In [105]:
import requests
import json
import time
import pandas as pd
import pickle

**Testing Pushshift API**

In [76]:
tr = requests.get('https://api.pushshift.io/reddit/search/submission/?subreddit=nottheonion')

In [77]:
tr.status_code

200

In [5]:
tr_js = tr.json()

In [7]:
tr_js['data'][0]

{'author': 'witnessofments',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_zqaxy',
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1535653355,
 'domain': 'mirror.co.uk',
 'full_link': 'https://www.reddit.com/r/nottheonion/comments/9bm5gg/easyjet_passengers_in_extraordinary_mass_brawl/',
 'id': '9bm5gg',
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_self': False,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comments': 1,
 'num_crossposts': 0,
 'over_18': False,
 'parent_whitelist_status': 'all_ads',
 'permalink': '/r/nottheonion/comments/9bm5gg/easyjet_passengers_in_extraordinary_mass_brawl/',
 'pinned': False,
 'pwls': 6,
 'retrieved_on': 1

In [8]:
tr_js['data'][0]['title']

"Easyjet passengers in extraordinary mass brawl 'after woman gets breasts out' and starts 'giving lapdances’"

In [5]:
post_list = []
post_list.extend(tr_js)

In [7]:
post_list

['data']

Seems to work well, I'll try and apply the same formula we used for the Reddit API

**Gathering Data**

Based off of the `total_results` key within pushshift API, the two subreddits that I'm looking at comparing contain the following number of posts:

- /r/theonion: 7124
- /r/notheonion: 115527

Given the size of both subreddits, I'll look to gather 5000 posts from each to analyze. That should give me enough data for my model

In [78]:
posts = []
before = None
for i in range(200):
    print(i)
    if before == None:
        params = {}
    else:
        params = {'before': before}
    url = 'https://api.pushshift.io/reddit/search/submission/?subreddit=nottheonion'
    res = requests.get(url, params=params)
    if res.status_code == 200:
        the_json = res.json()
        posts.extend(the_json['data'])
        before = the_json['data'][-1]['created_utc']
    else:
        print(res.status_code)
        break
    time.sleep(1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [34]:
before

1535140966

In [79]:
len(posts)

5000

In [80]:
import json
with open('data/nottheonion_ps_20180830.json', 'w') as fout:
    json.dump(posts, fout)

**Separating Titles from JSON and Saving into List**

In [81]:
title_list = []
c = 0
for i in range(len(posts)):
    title_list.append(posts[c]['title'])
    c += 1

In [82]:
len(title_list)

5000

In [83]:
len(set(title_list))

4086

In [84]:
df = pd.DataFrame(title_list)

In [85]:
df['sub'] = 1

In [86]:
df.rename(columns = {0:'Title'}, inplace=True)

**Getting Rid of Duplicate Values**

In [87]:
df.drop_duplicates(inplace=True)

**Working on /r/TheOnion**

In [88]:
posts_o = []
before = None
for i in range(200):
    print(i)
    if before == None:
        params = {}
    else:
        params = {'before': before}
    url = 'https://api.pushshift.io/reddit/search/submission/?subreddit=theonion'
    res = requests.get(url, params=params)
    if res.status_code == 200:
        the_json = res.json()
        posts_o.extend(the_json['data'])
        before = the_json['data'][-1]['created_utc']
    else:
        print(res.status_code)
        break
    time.sleep(1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [89]:
len(posts_o)

5000

In [91]:
import json
with open('data/theonion_ps_20180830.json', 'w') as fout:
    json.dump(posts_o, fout)

In [92]:
title_list_o = []
c = 0
for i in range(len(posts_o)):
    title_list_o.append(posts_o[c]['title'])
    c += 1

In [93]:
len(title_list_o)

5000

In [94]:
len(set(title_list_o))

4722

In [95]:
df_o = pd.DataFrame(title_list_o)

In [96]:
df_o['sub'] = 0

In [97]:
df_o.rename(columns = {0:'Title'}, inplace=True)

**Getting Rid of Duplicate Values**

In [98]:
df_o.drop_duplicates(inplace=True)

In [99]:
df.info(), df_o.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4086 entries, 0 to 4999
Data columns (total 2 columns):
Title    4086 non-null object
sub      4086 non-null int64
dtypes: int64(1), object(1)
memory usage: 95.8+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4722 entries, 0 to 4999
Data columns (total 2 columns):
Title    4722 non-null object
sub      4722 non-null int64
dtypes: int64(1), object(1)
memory usage: 110.7+ KB


(None, None)

**Saving Title Lists as a `pickle`**

In [108]:
with open("data/nto_t.pkl", 'wb') as nto_t:
    pickle.dump(df, nto_t)

In [109]:
with open("data/to_t.pkl", 'wb') as to_t:
    pickle.dump(df, to_t)